In [187]:
from qutip import *
from numpy import pi, linalg
from scipy.special import jv

In [188]:
si = qeye(2)

sx = sigmax()

sy = sigmay()

sz = sigmaz()

n = 2

m = 1

extmagfield_m = 1

extmagfield_n = [[0, 0, 0],[0, 0, 0],[0.3, 0.3, 0.3]]

JJ = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]

AA = [[0, 0, 0] ,[0, 0, 0], [0.99925, 0.625147, 0.551273]]
gama = [[0.0124425, 0.0806628, 0.00999575],[0.0550028, 0.0758354, 0.07346340],[0.0972069, 0.0723954, 0.07405450]]
# gama = 1
t = 1.77
E1 = (
    1 / 2 * sum([sum(i) for i in zip(*JJ)])
    + 1 / 2 * sum([sum(i) for i in zip(*gama)])
    + 1 / 2 * sum([sum(i) for i in zip(*AA)])
    + 1 / 2 * sum([sum(i) for i in zip(*extmagfield_n)])
    + 1 / 2 * extmagfield_m
)
tau = E1 * t / 2
print(E1)
kappa = 3 / 2 * tau
print(tau)

2.313364725
2.047327781625


In [189]:
def spin_op(N):
    """calculates spin oprators

    Args:
        N (int): number of electrons or atoms

    Returns:
        matrix: matrices of spin oprators
    """
    Sx = []
    Sy = []
    Sz = []

    for n in range(N):
        x_list = []
        y_list = []
        z_list = []

        for m in range(N):
            x_list.append(si)
            y_list.append(si)
            z_list.append(si)

        x_list[n] = sx
        y_list[n] = sy
        z_list[n] = sz

        Sx.append(tensor(x_list))
        Sy.append(tensor(y_list))
        Sz.append(tensor(z_list))

    return [Sx, Sy, Sz]

In [190]:
Sx, Sy, Sz = spin_op(m+n)
# HS & HB are bare hamiltonians of the central system and bath
# V is the system-bath interaction
HS = HB = V = 0
for i in range(m):
    HS += extmagfield_m * Sx[i] + extmagfield_m * Sy[i] + extmagfield_m * Sz[i]
    for j in range(m):
        HS += JJ[0][j] * Sx[i] * Sx[j] + JJ[1][j] * Sy[i] * Sy[j] + JJ[2][j] * Sz[i] * Sz[j]

for i in range(m,m+n):
    HB += extmagfield_n[0][i] * Sx[i] + extmagfield_n[1][i] * Sy[i] + extmagfield_n[2][i] * Sz[i]
    for j in range(m,m+n):
        HB += gama[0][j] * Sx[i] * Sx[j] + gama[1][j] * Sy[i] * Sy[j] + gama[2][j] * Sz[i] * Sz[j]
for i in range(m,m+n):
    for j in range(m):
        V += AA[0][j] * Sx[i] * Sx[j] + AA[1][j] * Sy[i] * Sy[j] + AA[2][j] * Sz[i] * Sz[j]
HH = HS + HB + V


In [191]:
HH





Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[ 4.13135715+0.j  0.        +0.j  0.        +0.j -0.05864025+0.j
   1.        -1.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  1.23995735+0.j  0.23995735+0.j  0.        +0.j
   0.        +0.j  1.        -1.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.23995735+0.j  1.23995735+0.j  0.        +0.j
   0.        +0.j  0.        +0.j  1.        -1.j  0.        +0.j]
 [-0.05864025+0.j  0.        +0.j  0.        +0.j -1.06564285+0.j
   0.        +0.j  0.        +0.j  0.        +0.j  1.        -1.j]
 [ 1.        +1.j  0.        +0.j  0.        +0.j  0.        +0.j
  -1.86564285+0.j  0.        +0.j  0.        +0.j -0.05864025+0.j]
 [ 0.        +0.j  1.        +1.j  0.        +0.j  0.        +0.j
   0.        +0.j -0.76004265+0.j  0.23995735+0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  1.        +1.j  0.        +0.j
   0.        +0.j  0.23995735+0.j 

In [192]:
def TG(k, G):
    if k == 1:
        return 1
    elif k == 2:
        return G
    else:
        return 2 * G * TG(k - 1, G) - TG(k - 2, G)

In [193]:
G = 2 * HH / E1
UU = 0
for k in range(1, int(kappa) + 1):
    a = 1
    if k == 1:
        a = 2
    UU += a * ((1j) ** k) * jv(k, tau) * TG(k, G)



In [194]:
UU

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = False
Qobj data =
[[-1.29761192-2.61071695j  0.        +0.j          0.        +0.j
   0.01841823+0.03670366j -0.77666872-0.14849164j  0.        +0.j
   0.        +0.j          0.02394461+0.02394461j]
 [ 0.        +0.j         -0.38945639+0.54904219j -0.07536785-0.1214934j
   0.        +0.j          0.        +0.j         -0.41207045+0.21610662j
  -0.09798192-0.09798192j  0.        +0.j        ]
 [ 0.        +0.j         -0.07536785-0.1214934j  -0.38945639+0.54904219j
   0.        +0.j          0.        +0.j         -0.09798192-0.09798192j
  -0.41207045+0.21610662j  0.        +0.j        ]
 [ 0.01841823+0.03670366j  0.        +0.j          0.        +0.j
   0.3347062 +0.6421498j   0.02394461+0.02394461j  0.        +0.j
   0.        +0.j         -0.28667206+0.34150501j]
 [ 0.14849164-0.77666872j  0.        +0.j          0.        +0.j
  -0.02394461+0.02394461j  0.58597703+0.1633764j   0.        +0.j
  

In [195]:
zero = basis(2) 
c = create(2)
one=c*zero
psi=tensor(tensor(one,basis(2)),basis(2))
psi0=psi*psi.dag()
print("trace: {}".format(psi0.tr())) 
psi0

trace: 1.0


Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]

In [196]:
# psif = UU * psi0
# result = psif * psif.dag()
# print("trace: {:.1f}".format(result.tr()))
# result
psif = UU * psi0 * UU.dag()
print("trace: {:.2f}".format(psif.tr()))
psif

trace: 1.00


Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[ 6.25264068e-01+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j         -2.21525999e-02+0.01504145j
  -4.79370063e-01+0.03987665j  0.00000000e+00+0.j
   0.00000000e+00+0.j         -1.26439025e-02-0.01142242j]
 [ 0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j        ]
 [ 0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j        ]
 [-2.21525999e-02-0.01504145j  0.00000000e+00+0.j
   0.00000000e+00+0.j          1.14668826e-03+0.j
   1.79429724e-02+0.01011901j  0.00000000e+00+0.j
   0.00000000e+00+0.j          1.73183648e-04+0.00070885j]
 [-4.79370063e-01-0.03987665j  0.00000000e+00+0.j
   0